<a href="https://colab.research.google.com/github/rohandawar/Databricks/blob/main/Breast_Cancer_Clasification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, I am checking the Breast_Cancer_Classification

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 12.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=2aebb000351e080aa012cbfaf95cf016b182045dac17f13833b0c9ac645d256b
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [4]:
# Import Libs
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator 

# Pandas
import pandas as pd
from sklearn.datasets import load_breast_cancer

In [6]:
# Read the dataset
cancer_data = load_breast_cancer()

In [7]:
# create a pandas dataframe
df = pd.DataFrame(cancer_data.data, columns=cancer_data.feature_names)
df['label'] = cancer_data.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,label
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [10]:
# Create a spark session
spark = SparkSession.builder.appName('RandomForestClassifierExample').getOrCreate()

# Convert the pandas dataframe to a spark dataframe
spark_df = spark.createDataFrame(df)
spark_df.head(1)

[Row(mean radius=17.99, mean texture=10.38, mean perimeter=122.8, mean area=1001.0, mean smoothness=0.1184, mean compactness=0.2776, mean concavity=0.3001, mean concave points=0.1471, mean symmetry=0.2419, mean fractal dimension=0.07871, radius error=1.095, texture error=0.9053, perimeter error=8.589, area error=153.4, smoothness error=0.006399, compactness error=0.04904, concavity error=0.05373, concave points error=0.01587, symmetry error=0.03003, fractal dimension error=0.006193, worst radius=25.38, worst texture=17.33, worst perimeter=184.6, worst area=2019.0, worst smoothness=0.1622, worst compactness=0.6656, worst concavity=0.7119, worst concave points=0.2654, worst symmetry=0.4601, worst fractal dimension=0.1189, label=0)]

In [11]:
# Perform the train Test split
train_df, test_df = spark_df.randomSplit([0.7,0.3], seed=100)


In [12]:
# Assemble the features into Vectors 
assembler = VectorAssembler(inputCols=cancer_data.feature_names, outputCol='Features')

# Tranform the data
train_df = assembler.transform(train_df)
test_df = assembler.transform(test_df)

In [13]:
# Train the random forest Classfier
rf = RandomForestClassifier(labelCol='label', featuresCol='Features')
model = rf.fit(train_df)

In [14]:
# Make the Predictions
predictions = model.transform(test_df)

In [16]:
# Evaluate the model's performance using a binary class Classifier evaluator
evaluator = BinaryClassificationEvaluator(labelCol='label', rawPredictionCol='prediction')
accuracy = evaluator.evaluate(predictions)
print('Accuracy:', accuracy)

Accuracy: 0.9118639667705088
